<h1>Lightning DataModule</h1>

<p>A LightningDataModule is simply a collection of PyTorch DataLoaders with the corresponding transforms and downloading/processing steps required to prepare the data in a reproducible fashion. It encapsulates all steps requires to process data in PyTorch:</p>

-   Download and tokenize,
-   Clean and save to disk
-   Load inside Dataset
-   Apply transforms
-   Wrap inside a DataLoader

<p>Importantly, a LightningDataModule is shareable and reusable. That is, it - -  centralizes all data preparation tasks to have a self-contained object that can exactly duplicate a dataset via identical splits and transforms.</p>

In [ ]:
import os
import torch
import lightning.pytorch as pl

<p>Let’s create our MNIST LightningDataModule. First, we initialize with some relevant parameters and create the transform object that we will use to process our raw data. Lastly, we create a dictionary which we will pass into our DataLoaders later.<p>

In [ ]:
class MNISTDataModule(pl.LightningDataModule):
    
    def __init__(self, data_dir="./data", batch_size=128, num_workers=int(os.cpu_count() / 2)):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers

        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1,), (0.3,))
        ])

        self.dl_dict = {'batch_size': self.batch_size, 'num_workers': self.num_workers}


<p>Next, we define the prepare_data() function. It defines how to download and e.g. tokenize data. Lightning ensures that this method is called with a single process to avoid corrupted data. Here, we simply download our training and testing sets.</p>

In [ ]:
    def prepare_data(self):
        datasets.MINST(self.data_dir, train=True, download=True)
        datasets.MINST(self.data_dir, train=False, download=True)

<p>Next up is the setup() function. It defines data operations you might want to perform on every GPU. Use it to define datasets and e.g. split/transform data or build a vocabulary. In our case, we will split the data into training, validation, and testing sets, using our transform defined in the class __init__() function.</p>

In [ ]:
    def setup(self,stage=None):
        # Validation data not strictly necessary for GAN but added for completeness

        if stage == "fit" or stage is None:

            mnist_full = datasets.MNIST(self.data_dir, train=True, transform=self.transform)
            self.mnist_train, self.mnist_val = random_split(mnist_full, 55000, 5000)
        
        if stage == "test" or stage is None:
            self.mnist_test = datasets.MNIST(self.data_dir, train=False, trainsform=self.transform)

<p>Finally, we define training, validation, test, and predict DataLoaders. Usually the datasets defined in setup() are simply wrapped in DataLoader objects. Note that validation data is not really necessary for GANs given their unusual evaluation protocols, but nevertheless val_dataloader() is added here for completeness. Further, we have not defined a predict DataLoader here, but the process is identical.</p>

In [ ]:
    ## For dataloaders, usually just wrap dataset defined in setup

    def train_dataloader(self):
        return DataLoader(self.mnist_train, **self.dl_dict)

    def val_dataloader(self):
        return DataLoader(self.mnist_train, **self.dl_dict)

    def test_dataloader(self):
        return DataLoader(self.mnist_train, **self.dl_dict)

<h1>Building the Lightning Module</h1>

<p>Recall from above that the central object in the Lightning workflow is the LightningModule, which encapsulates the entire model ecosystem. Now that we’ve finished defining our LightningDataModule object, we can build our LightningModule. In our case, this ecosystem includes two models - the generator and the discriminator. We’ll define them now, noting that the process is identical to vanilla PyTorch.</p>

<h1>Building the Discriminator</h1>

<p>First, we’ll construct our discriminator as an nn.Module. We’ll use a simple CNN with two convolutional layers followed by a fully connected network to map from 28x28 single channel digit images to classification predictions. Remember, the purpose of the discriminator is to classify images as real or fake, so we only need a single output node, in contrast to the ten required for the digit classification networks often built for MNIST.</p>

In [ ]:
class Discriminator(nn.Module):
    
    def __init__(self):
        
        # Simple CNN
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 1)

<p>Next, we define the forward pass for the discriminator. We use max poolings with a kernel size of two followed by ReLU for the convolutional layers, and use sigmoid for the final activation</p>

In [ ]:
    def forward(self, x):

            x = F.relu(F.max_pool2d(self.conv1(x), 2))
            x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))

            # Flatten the tensor so it can be fed into the FC layers
            x = x.view(-1, 320)
            x = F.relu(self.fc1(x))
            x = F.dropout(x, training=self.training)
            x = self.fc2(x)
            
            return F.sigmoid(x)

<h1>Building the Generator</h1>
<p>
Similarly, we construct the generator as an nn.Module. We input data points from a latent space which feed into a linear layer to provide us with enough nodes to create 7x7 images with 64 feature maps. We then used transposed convolutions for learnable upsampling, ultimately collapsing the data to a 28x28 single channel image (i.e. the digit image) via a final convolutional layer.
</p>

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim):

        super().__init__()
        
        self.lin1 = nn.Linear(latent_dim, 7*7*64)
        self.ct1 = nn.ConvTranspose2d(64, 32, 4, stride=2)
        self.ct2 = nn.ConvTranspose2d(32, 16, 4, stride=2)
        self.conv = nn.Conv2d(16, 1, kernel_size=7)

<p>And again, we define the forward pass.</p>

In [ ]:
    def forward(self, x):
            # Pass latent space input into linear layer and reshape
            x = self.lin1(x)
            x = F.relu(x)
            x = x.view(-1, 64, 7, 7)

            
            # Transposed convolution to 16x16 (64 feature maps)
            x = self.ct1(x)
            x = F.relu(x)
            
            # Transposed convolution to 34x34 (16 feature maps)
            x = self.ct2(x)
            x = F.relu(x)
            
            # Convolution to 28x28 (1 feature map)
            return self.conv(x)

<h1>
Defining the GAN
</h1>
<p>
Now that we have our traditional PyTorch nn.Module models, we can build our LightningModule. This is where we will see the Lightning approach diverge from the vanilla PyTorch approach.
</p>

<h1>
Initialization
</h1>
<p>
First, we define our initialization function, inputting our latent dimensionality as well as a learning rate and betas for our Adam optimizers. save_parameters() allows us to store these arguments under the self.hparams attribute (also stored in the model checkpoint) for easier reinstantiation after training. Finally, we initialize the generator and discriminator models within our ecosystem, and generate latent space points that we can use to monitor progress. These points will give us a consistent set of data to track how the generator is progressing in mapping from the same set of latent points to images as it learns.
</p>

In [ ]:
class GAN(pl.LightningModule):
    ## Initialize. Define latent dim, learning rate, and Adam betas 
    def __init__(self, latent_dim=100, lr=0.0002, 

                 b1=0.5, b2=0.999, batch_size=128):
        super().__init__()
        self.save_hyperparameters()

        self.generator = Generator(latent_dim=self.hparams.latent_dim)
        self.discriminator = Discriminator()

        self.validation_z = torch.randn(8, self.hparams.latent_dim)

<h1>
Forward Pass and Loss Function
</h1>
<p>
Next, we define the GAN’s forward pass and loss function. Note that using self.generator(z) is preferred over self.generator.forward(z) given that the forward pass is only one component of the calling logic when self.generator(z) is called.

We will be using the loss function in two different ways - one as the discriminator loss and one as the generator loss. The first way is to update the discriminator as a classifier in the canonical fashion, inputting real and generated images with the appropriate labels. The second way is to update the generator using the loss from the discriminator on generated images. That is, the better the discriminator is at detecting fake images, the more the generator is updated. More on this later.
</p>

In [ ]:
    def forward(self, z):
        return self.generator(z)

    def adversarial_loss(self, y_hat, y):
        return F.binary_cross_entropy(y_hat, y)

<h1>
Training Step
</h1>
<p>
Next up is defining what occurs during a training step for the GAN. If we are training the generator, we generate images and then get predictions on them via the discriminator. When we calculate the loss, we are importantly using deceptive labels here. That is, despite the fact that the images are fabricated, we label them as real in the loss function. This is because we want them to be classified as real, so the deceptive labels will lead to greater loss when the generated images are (correctly) classified as fake. We also log some of the fabricated images to be viewed in TensorBoard, and output relevant values.

There are no special details for training the discriminator. In a very straightforward fashion, we compute the loss on the real images and the fake images (with honest labels) and average them as the discriminator loss. Again we output the relevant parameters.
</p>

In [ ]:
    def training_step(self, batch, batch_idx, optimizer_idx):
        real_imgs, _ = batch

        # sample noise
        z = torch.randn(real_imgs.shape[0], self.hparams.latent_dim)

        # train generator
        if optimizer_idx == 0:
            self.generated_imgs = self(z)
            predictions = self.discriminator(self.generated_imgs)
            g_loss = self.adversarial_loss(predictions, torch.ones(real_imgs.size(0), 1))
            

            # log sampled images
            sample_imgs = self.generated_imgs[:6]
            grid = torchvision.utils.make_grid(sample_imgs)
            self.logger.experiment.add_image("generated_images", grid, 0)


            tqdm_dict = {"g_loss": g_loss}
            output = OrderedDict({"loss": g_loss, "progress_bar": tqdm_dict, "log": tqdm_dict})
            return output

        # train discriminator
        if optimizer_idx == 1:
            real_preds = self.discriminator(real_imgs)
            real_loss = self.adversarial_loss(real_preds, torch.ones(real_imgs.size(0), 1))

            fake_preds = self.discriminator(self(z).detach())
            fake_loss = self.adversarial_loss(fake_preds, torch.zeros(real_imgs.size(0), 1)) 


            d_loss = (real_loss + fake_loss) / 2
            tqdm_dict = {"d_loss": d_loss}
            output = OrderedDict({"loss": d_loss, "progress_bar": tqdm_dict, "log": tqdm_dict})
            return output

<h1>
Configure Optimizers
</h1>
<p>
Now we can configure our Adam optimizers using the learning rates and betas we saved in self.hparams. We configure one optimizer for the generator and one for the discriminator.
</p>

In [ ]:
    def configure_optimizers(self):
        lr = self.hparams.lr
        b1 = self.hparams.b1
        b2 = self.hparams.b2

        opt_g = torch.optim.Adam(self.generator.parameters(), lr=lr, betas=(b1, b2))
        opt_d = torch.optim.Adam(self.discriminator.parameters(), lr=lr, betas=(b1, b2))
        return [opt_g, opt_d], []

<h1>
Epoch End
</h1>
<p>
Finally, we define the on_epoch_end() method. It is not strictly necessary, but we use it here to log images so that we can observe training progress across epochs. It is called whenever a training, testing, or validation epoch ends. Note that each of the training, testing, validation, and predict cases have their own epoch_end() functions in case you do not want to perform such a function across the board!
</p>

In [ ]:
    def on_epoch_end(self):
        # log sampled images
        sample_imgs = self(self.validation_z)
        grid = torchvision.utils.make_grid(sample_imgs)
        self.logger.experiment.add_image("generated_images", grid, self.current_epoch)

<h1>
Program Code
</h1>
<p>
Now we can write the main program code to utilize all of the components we’ve defined above, which requires just a few simple lines with Lightning.
</p>

<h1>
Lightning Trainer
</h1>
<p>
Recall from above that the LightningModule is not an abstraction layer on top of PyTorch, but simply a reorganization of code. The abstraction in Lightning comes from the Trainer class.  The Trainer class has very straightforward minimal usage, and is the source of many benefits, including:
</p>

-   The ability to override any automation component
-   The omission of hardware references
-   The removal of boilerplate code
-   The inclusion of under-the-hood best practices via contributors from top AI labs

<p>
For our purposes, we simply need to pass in a value for the maximum number of training epochs.
</p>

In [ ]:
trainer = pl.Trainer(max_epochs=20)

<h1>
GAN Training
</h1>
<p>
We’ve defined our LightningDataModule and LightningModule above and instantiated the trainer which will operate on our LightningModule. Now all we need to do is instantiate our LightningDataModule and LightningModule and pass them in to the trainer!
</p>

In [ ]:
dm = MNISTDataModule()
model = GAN()
trainer.fit(model, dm)